In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-126677")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-126677
Azure region: southcentralus
Subscription id: 4910dccd-0348-46c4-a51f-d8c85e078b14
Resource group: aml-quickstarts-126677


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
compute_name = 'aml-cluster'

compute_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_V2", 
                                                       min_nodes = 0,
                                                       max_nodes = 4)
#Creating the compute target
aml_compute = ComputeTarget.create(ws, compute_name, compute_config)


### YOUR CODE HERE ###
print(aml_compute.get_status().serialize())

aml_compute.wait_for_completion(show_output= True)

{'errors': [], 'creationTime': '2020-11-17T20:47:20.378480+00:00', 'createdBy': {'userObjectId': 'a056b403-3b04-43b1-b486-cbb94f90e96a', 'userTenantId': '660b3398-b80e-49d2-bc5b-ac1dc93b5254', 'userName': None}, 'modifiedTime': '2020-11-17T20:50:22.724574+00:00', 'state': 'Running', 'vmSize': 'STANDARD_DS3_V2'}

Running


In [4]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

aml-cluster ComputeInstance Succeeded


In [76]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
from azureml.train.hyperdrive import choice

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        "--C": uniform(0.5, 2), "--max_iter": choice(10,20,50,100,200)
    }
)
### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor= 0.1, delay_evaluation = 5)
### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = '.', compute_target = aml_compute, 
              vm_size= "Standard_D2_V2",
              entry_script= 'train.py') ### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hd_config = HyperDriveConfig(estimator = est,
                                    hyperparameter_sampling =ps,
                                    policy = policy,
                                    primary_metric_name = "Accuracy",
                                    primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs= 20,
                                    max_concurrent_runs = 4)### YOUR CODE HERE ###

In [77]:
hyperdrive_run= exp.submit(config= hd_config)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


In [78]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###